<center>
<h1> D ONE MLOps AWS BlogPost </h1>
    <h2>Work In Progress</h2>
<hr>
<h1>Random Forest Trainig Pipeline</h1>
<hr>
 </center>

<h2>Problem Introduction</h2>

* See EDA notebook
* Add a few high level lines here

# Setting up shop

A known challenge with SageMaker Studio default libraries is that they are often outdated. In this case, we want to make sure that pandas and s3fs is up-to-date. s3fs is a file system interface for S3 (https://pypi.org/project/s3fs/).

In [3]:
! pip install pandas s3fs --upgrade

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


# Import packages
We need a few packages, let's import them.

In [40]:
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta

import boto3  # Amazon Web Services (AWS) Software Development Kit (SDK) for Python
import os
import pandas as pd
import sagemaker

# Set up global variables

In [13]:
# This is the role that SageMaker assumes to leverage AWS resources
ROLE = sagemaker.get_execution_role()
ROLE

'arn:aws:iam::451811961115:role/AmazonSageMaker-ExecutionRole-20200526T075798'

In [14]:
# Paths for the containers that we will create
# S3 Bucket where the data is stored
BUCKET_NAME = "aws-sagemaker-blogpost"
BUCKET = f's3://{BUCKET_NAME}'

# Raw data paths
RAW_DATA_FOLDER = 'data'
RAW_DATA_FILE = 'wind_turbines.csv'
RAW_DATA_PATH = os.path.join(BUCKET, RAW_DATA_FOLDER, RAW_DATA_FILE)

# Path where the processed objects will be stored
now = datetime.now() # get current time to ensure uniqueness of the output folders
PROCESSED_DATA_FOLDER = 'processed_' + now.strftime("%Y-%m-%d_%H%M_%S%f")
PROCESSED_DATA_PATH = os.path.join(BUCKET, PROCESSED_DATA_FOLDER)

# Paths for model train, validation, test split
TRAIN_DATA_PATH = os.path.join(PROCESSED_DATA_PATH, 'train.csv')
VALIDATION_DATA_PATH = os.path.join(PROCESSED_DATA_PATH, 'validation.csv')
TEST_DATA_PATH = os.path.join(PROCESSED_DATA_PATH, 'test.csv')

# Path to model output data
MODEL_OUTPUT = os.path.join(BUCKET, 'output')

# Preprocess data (TODO)

The data comes in chunks, but for our purposes, we would like to have one large file with data. We could also extend the code to read only new data and add it to the file, but for simplicity, let's load the whole dataset for processing.

Note: Having the data in chunks would allow for online processing of new data when it comes in, e.g., when a turbine reports the next couple of months data.

For details on the dataset, see the exploratory data analysis work in Part 1.

* TODO: Expand
* TODO: Link to EDA.

In [76]:
from preprocessing import get_train_test_split
from transforming import select_and_transform_data

df = pd.read_csv(RAW_DATA_PATH)
get_train_test_split(df=df, n_days_test=10);
_FEATURES = ['wind_speed', 'power', 'nacelle_direction', 'wind_direction',
            'rotor_speed', 'generator_speed', 'temp_environment',
            'temp_hydraulic_oil', 'temp_gear_bearing', 'cosphi',
            'blade_angle_avg', 'hydraulic_pressure']
x, y = select_and_transform_data(df=df, features=features)
x.shape, y.shape

((46286, 12), (46286,))

In [79]:
df['categories_sk'].value_counts()

5.0     2415
20.0     235
3.0      175
19.0     123
24.0      10
22.0       9
Name: categories_sk, dtype: int64

In [78]:
y.value_counts()

9    46286
Name: categories_sk, dtype: int64

In [38]:
min_date = df['date'].min()
max_date = df['date'].max()

In [43]:
test_dates = [datetime.strftime(max_date - timedelta(days=i), '%Y-%m-%d') for i in range(3)]
test_dates

['2020-03-31', '2020-03-30', '2020-03-29']

In [71]:
select_and_transform_data(df, features)

(       wind_speed     power  nacelle_direction  wind_direction  rotor_speed  \
 0        0.422840  0.117495         263.698129      262.272441     0.672849   
 1        0.455156  0.133970         263.999743      264.661897     0.696582   
 2        0.506924  0.179506         272.100926      270.700148     0.770216   
 3        0.527410  0.226506         274.385447      274.586154     0.806076   
 4        0.545956  0.222543         281.036740      277.383938     0.823141   
 ...           ...       ...                ...             ...          ...   
 52378    0.542102  0.298023         235.076796      228.789722     0.852574   
 52379    0.469812  0.196782         228.616130      225.171532     0.775082   
 52380    0.492456  0.234606         230.451366      225.720881     0.791443   
 52381    0.508890  0.281787         227.701908      232.073361     0.804596   
 52382    0.588925  0.399555         225.827208      232.655443     0.887138   
 
        generator_speed  temp_environm